In [1]:
import requests
from bs4 import BeautifulSoup

headers = {
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
}
res = requests.get('http://movie.mtime.com/251525/reviews', headers=headers)
soup = BeautifulSoup(res.text, 'html.parser')
comments = soup.select('ul.list > li > div.contentBox > p.content')

for comment in comments:
  print(comment.text)

那么，为什么在网页开发者工具中有评论的数据，而网页源代码中就没有呢？

那这种实时改变的数据是怎样展现到我们面前的呢？这就是我们今天课程的重点——网页可以通过 API 获取数据，实时更新内容。API 即应用程序接口，它规定了网页与服务器之间可以交互什么数据、通过什么样的方式进行交互。

为了找到评论的真正藏身之地，我们得先来看看开发者工具中的 Network。

我们还可以点击每一个请求，查看每个请求的详细信息。详细信息里的 Response（响应）里是服务器返回的内容。第一个请求是网页，它的响应是网页源代码，即我们使用 requests.get() 获取到的 res.text 内容。

因为一次性加载整个网站很慢，为了提升网页加载速度，有些网站将网站的骨架和内容拆分开，加载骨架后再通过多个请求获取内容，最终组成完整的网站。而有些老的网站或轻量级网站，仍然是一次性返回整个网站的内容，比如豆瓣。

### 什么是 XHR

XHR 全称 XMLHttpRequest，是浏览器内置的对象。浏览器想要在不刷新网页前提下加载、更新局部内容时，必须通过 XHR 向存放数据的服务器发送请求。

接下来，跟上我的操作，我们一起在 XHR 中寻找评论。

首先点击 Network 中的 XHR 过滤其他类型的请求。可以看到，仍然有很多的请求，我们要从中找到评论数据。笨一点的方法是一条一条地查看，直到找到评论的数据，但这样耗时又耗力。

聪明一点的方法是通过名称来找，既然是评论的数据，请求名称中可能会带有 comment（评论），这样我们就能极大地缩小查找范围。

还有一个小技巧是：由于数据是分页加载，我们可以先将请求记录清空，再点击下方分页导航的第 2 页，这样，加载第 2 页评论的请求说不定就会被我们“守株待兔”逮个正着。

从图中我们可以看到，时光网规定了需要通过 GET 方法，向 Request URL 发送请求。点击第二页查看电影评论的行为，实际上是浏览器帮我们自动填充查询参数，向时光网获取短影评数据，参数详情为：

id 为 251525 (movieId=251525) ；
第 2 页内容 (pageIndex=2)；
每页 20 条 (pageSize=20)。
遵循某种规则向指定 URL 发送请求，从而获得相应数据的过程，就是 通过 API 获取数据。

下面的 Status Code 是状态码，之前说过，如果是 200，表示成功。其他信息我们无需关注。

另外，为了防止被网站反爬，我们还要在 Headers 中观察一下 Request Headers（请求头），有两个参数要注意。

user-agent 我们之前已经说过，用途是将爬虫伪装成浏览器；另外一个 referer 字段，字面意思是“发起者，发送人”，用来验证这个请求的发起方是否合法。也就是说，服务器要验证这个请求是由谁发出的，只接受从特写网页上发出该请求，比如这里就是时光网的网址。如果这个字段不加，可能会爬取失败。

In [2]:
import requests

headers = {
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36',
  'referer': 'http://movie.mtime.com/'
}
res = requests.get('http://front-gateway.mtime.com/library/movie/comment.api?tt=1641893701852&movieId=251525&pageIndex=2&pageSize=20&orderType=1', headers=headers)
print(res.text)

{"code":0,"data":{"count":3317,"hasMore":true,"list":[{"nickname":"瀛26000","userImage":"https://img2.mtime.cn/u/285/2016285/7c677c7f-4503-42e9-a63f-683e9f97148c/128X128.jpg","rating":"8.5","content":"这是我看过的最有血有肉有人情味还紧跟时代潮流的一版哪吒了，没有金吒木吒只有哪吒，李靖没有让人讨厌的琵琶精小老婆，敖丙也不是奸淫掳掠无恶不作的龙二代，申公豹的口吃设计也承包了一部分笑点，太乙真人的火锅味","memberLevel":1,"isTicket":false,"userRelation":0,"praiseDownCount":0,"isPraiseDown":false,"userId":2016285,"attitude":0,"praiseCount":9,"isPraise":false,"commentCount":0,"commentTime":1564207492,"mvpType":1,"commentId":225032751,"title":null,"isWantSee":false,"location":null},{"nickname":"王奋斗","userImage":"http://img5.mtime.cn/up/2017/05/05/090951.70534399_128X128.jpg","rating":"1.0","content":"暴力狂，咒骂狂，虐待狂，嘲弄狂，自大狂，却被疯狂地拥趸与吹捧，只因为这些是内心充满狂暴而狂暴又被彻底压抑的人群。通过丑陋的主角、蠢笨的配角，释放了长期被压抑的狂暴，既获得了精神上的胜利，又获得了发泄后","memberLevel":2,"isTicket":false,"userRelation":0,"praiseDownCount":0,"isPraiseDown":false,"userId":3775866,"attitude":0,"praiseCount":8,"isPraise":false,"commentCount":0,"commentTime":1568783360,"

链接中的 tt=1641893701852&movieId=251525&pageIndex=2&pageSize=20&orderType=1，可以拆分成一个字典：

In [ ]:
params = {
  "tt": "1641893701852",
  "movieId": "251525",
  "pageIndex": "2",
  "pageSize": "20",
  "orderType": "1"
}

res = requests.get(
  'http://front-gateway.mtime.com/library/movie/comment.api',
  params=params,
  headers=headers
)

五个参数中，movieId pageIndex pageSize 的意义我们可以根据字面意思猜到，应该分别代表 电影在时光网中的 ID，评论的第x页 和 每页评论数。

电影的页面地址  中的 251525 和 movieId 的值一致，可以作为佐证。

那剩下的 orderType 和 tt 是什么？

orderType 字面意思是排序方式，而我们发现，短影评页的右上方的确是有这个选项的。值为 1 代表的应该就是按最热排序。

剩下的 tt 参数，如果对编程不熟悉，可能毫无头绪。老师只能通过经验判断可能是 Unix 时间戳，依据有三个：

以 15、16、17 开头，是长整数，与当下时间戳的开头值相符；
字母 t 本身也可能和 time（时间）有关；
多次尝试刷新网页，重复发起请求，发现 tt 的值每次都在变大，非常像时间的增长。
提示：Unix 时间戳是指格林威治时间 1970 年 01 月 01 日 00 时 00 分 00 秒起至现在的总秒数。在 Python 基础课第 22 关已经学过了相关模块 time 的使用，同学们可以复习一下。

老师在分析网页时，请求中的 tt 的值是 1641893701852（到你学的时候数值肯定又不一样了），为了验证这个数字是不是时间戳，老师试着调用 time 模块，打印了下当前的时间戳，两两对比：

In [3]:
import time
print(time.time())
# 输出：1641896195.016945

1657684062.7475836


两个数字的前几位几乎一模一样！那几乎可以肯定 tt 就是时间戳了。我们知道，python 打印出的当前时间是以秒为单位，整数部分一共 10 位，而 tt 的值 1641893701852 是 13 位，那单位可能是毫秒。

果然，就是老师当时解析网页请求的时间。

获取评论的请求为什么要带上当前的时间戳？具体原因老师也不知道，可能是为了验证请求是否过期——如果当前时间戳与服务器时间差距过大，服务器可能认为是非法的过期请求，从而拒绝返回结果（猜想而已）。

In [4]:
import requests
import time

headers = {
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36',
  'referer': 'http://movie.mtime.com/'
}

params = {
  # 将当前时间戳转为毫秒后取整，作为 tt 的值
  "tt": "{}".format(int(time.time() * 1000)),
  "movieId": "251525",
  "pageIndex": "2",
  "pageSize": "20",
  "orderType": "1"
}

res = requests.get(
  'http://front-gateway.mtime.com/library/movie/comment.api',
  params=params,
  headers=headers)
print(res.text)

{"code":0,"data":{"count":3317,"hasMore":true,"list":[{"nickname":"瀛26000","userImage":"https://img2.mtime.cn/u/285/2016285/7c677c7f-4503-42e9-a63f-683e9f97148c/128X128.jpg","rating":"8.5","content":"这是我看过的最有血有肉有人情味还紧跟时代潮流的一版哪吒了，没有金吒木吒只有哪吒，李靖没有让人讨厌的琵琶精小老婆，敖丙也不是奸淫掳掠无恶不作的龙二代，申公豹的口吃设计也承包了一部分笑点，太乙真人的火锅味","memberLevel":1,"isTicket":false,"userRelation":0,"praiseDownCount":0,"isPraiseDown":false,"userId":2016285,"attitude":0,"praiseCount":9,"isPraise":false,"commentCount":0,"commentTime":1564207492,"mvpType":1,"commentId":225032751,"title":null,"isWantSee":false,"location":null},{"nickname":"王奋斗","userImage":"http://img5.mtime.cn/up/2017/05/05/090951.70534399_128X128.jpg","rating":"1.0","content":"暴力狂，咒骂狂，虐待狂，嘲弄狂，自大狂，却被疯狂地拥趸与吹捧，只因为这些是内心充满狂暴而狂暴又被彻底压抑的人群。通过丑陋的主角、蠢笨的配角，释放了长期被压抑的狂暴，既获得了精神上的胜利，又获得了发泄后","memberLevel":2,"isTicket":false,"userRelation":0,"praiseDownCount":0,"isPraiseDown":false,"userId":3775866,"attitude":0,"praiseCount":8,"isPraise":false,"commentCount":0,"commentTime":1568783360,"

In [5]:
print(type(res.text)) #所以，从打印结果上看，res.text 是多层级的字典吗？并不是，只是长得像字典的字符串罢了，我们可以验证一下
# 输出：<class 'str'> 

<class 'str'>


这种长得像字典的字符串，是一种名为 JSON 的数据格式。我们需要将其转换成真正的 字典/列表，才能从中提取出评论数据。所以，接下来我们学习 JSON 来将其转换成字典/列表。

### 什么是 JSON

JSON（JavaScript Object Notation）是一种轻量级的数据交换格式。 易于人阅读和编写，同时也易于机器解析和生成。

JSON 建构于两种结构：键值对的集合 和 值的有序列表，分别对应 Python 里的字典和列表，这些都是常见的数据结构。大部分现代计算机语言都支持 JSON，所以 JSON 是在编程语言之间通用的数据格式。

JSON 本质上就是一个字符串，只是该字符串符合特定的格式要求。也就是说，我们将字典、列表等用字符串的形式写出来就是 JSON，就像下面这样：

In [ ]:
# 字典
dict = {'price': 233}

# JSON
json = '{"price": 233}'

# 列表
list = ['x', 'y', 'z']

# JSON
json = '["x", "y", "z"]'
Tips：Python 字符串使用单引号或双引号没有区别，但 JSON 中，字符串必须使用英文的双引号来包裹。

你可能会有疑问，为什么不直接写成字典或列表，而非要写成 JSON 呢？这是因为不同编程语言的数据结构是不一样的，Python 的字典和列表在别的语言中可能并不写成这样。

而 JSON 是一种标准，规定了基本数据结构的写法，不同的编程语言拿到后解析成自己对应的数据结构即可。JSON 就像普通话，而不同语言的数据结构是方言。普通话是大家都能听懂的，而方言不能。

### 如何解析 JSON

JSON 响应内容的介绍如下，通过调用 json() 方法即可对响应内容解码，当然内容必须得是 JSON 格式的，否则将会报错。

In [6]:
import requests
import time

headers = {
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36',
  'referer': 'http://movie.mtime.com/'
}
params = {
  # 将当前时间戳转为毫秒后取整，作为 tt 的值
  "tt": "{}".format(int(time.time() * 1000)),
  "movieId": "251525",
  "pageIndex": "2",
  "pageSize": "20",
  "orderType": "1"
}

res = requests.get(
  'http://front-gateway.mtime.com/library/movie/comment.api',
  params=params,
  headers=headers)
  
print(type(res.json()))
# 输出：<class 'dict'>

<class 'dict'>


In [11]:
result = res.json()
print(result['data']['list'][0]['content'])
print(result['data']['list'][0]['nickname'])

这是我看过的最有血有肉有人情味还紧跟时代潮流的一版哪吒了，没有金吒木吒只有哪吒，李靖没有让人讨厌的琵琶精小老婆，敖丙也不是奸淫掳掠无恶不作的龙二代，申公豹的口吃设计也承包了一部分笑点，太乙真人的火锅味
瀛26000


In [13]:
import requests
import time

headers = {
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36',
  'referer': 'http://movie.mtime.com/'
}
params = {
  # 将当前时间戳转为毫秒后取整，作为 tt 的值
  "tt": "{}".format(int(time.time() * 1000)),
  "movieId": "251525",
  "pageIndex": "1",
  "pageSize": "20",
  "orderType": "1"
}

res = requests.get(
  'http://front-gateway.mtime.com/library/movie/comment.api',
  params=params,
  headers=headers)

comment_list = res.json()['data']['list']
for i in comment_list:
  print("用户：", i['nickname'])
  print("评论：", i['content'])

用户： 二哈排骨精
评论： 10年磨一剑，干翻迪斯尼
用户： JackieChen
评论： 一般般的动画电影，水军吹捧过头就不好了。真当有些人看了那么多年美漫和日漫 白看的？真当我们没见识？
用户： 麻匪118
评论： 中国式家庭典型代表，全村人的希望敖丙和我儿子再淘气也小可爱哪吒，史上最人性李靖，最可怜龙王，最接地气太乙金仙，最像男孩子的哪吒，哈哈哈，完美，必须二刷！
用户： 龚大叔
评论： 那些觉得不好看的，我只有用四川话回你：你晓得个锤子
用户： 天狼星上的眼镜猴
评论： 好好教育孩子，否则天雷来劈渣儿的时候都没人帮忙
用户： 银瞳
评论： 厅不错 音效画面都好 也没有小孩吵 电话响 问剧情 在国内看动画很久没有这么好的观影体验了 武戏分镜动作设计可圈可点 特别是江山图里四个人冻成冰球的段落很惊喜 可惜文戏让人如坐针毡 开场不到10分钟段
用户： jass
评论： 中国动画史上最悲怆的一幕：哪吒愤而举剑，割喉自杀。 今天的哪吒喊着 我命由我不由天，有点为赋新词强说愁的意思。艺术作品脱离了大时代背景，永远不可能成为真正意义上的经典。
用户： 心亦恋
评论： 休说苍天不由人，我命由我不由天。 视觉盛宴催旧泪，国漫希望谱新篇。
用户： 路见很平
评论： 都在说好，看了不过如此，风格不统一，过于糅杂，台词也尬得要命，从头到尾不舒服看不下去，编剧想让NPC怎样就怎样，完全不管人的行事逻辑，就最后的特效还行。总体不如同门的大圣归来，更不如彩条屋最出彩的大护
用户： 苗乐乐MiuMiu
评论： 每次大热电影出现都是言过其实了，跟风吹的人不必跟风黑的人少。故事上基本是胡编乱造毁名著，情感上可取的地方是没有过分煽情，但又有时幽默的不是地方，情感的拿捏比电影技术本身难多了。这个电影放到好莱坞水准衡
用户： 浪子回头金不换
评论： 前两天有位好友推荐看，加上这两天多方媒体好评报导，特别是时光网评分达到8.4，好奇释然，去了影院一观。看到了国产动画的进步，但是3D效果没做好，达不到真正的3D效果。剧情普通，没有惊喜感，从头看到尾令
用户： M_1504121653263072768
评论： 没想到一年多之后重返电影院看的第一部电影是国产动画片，还是一部这么好看的片子，好看到什么程度呢，我刚走出影厅，听到隔壁影厅哪吒的声音，我居然想溜进去再看一遍！整个故事情节编的好“圆”，没有觉得一丝

爬取多页

In [15]:
import requests
import time

headers = {
  'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36',
  'referer': 'http://movie.mtime.com/'
}

for num in range(1, 6):
  params = {
    "tt": "{}".format(int(time.time() * 1000)),
    "movieId": "251525",
    "pageIndex": "{}".format(num),
    "pageSize": "20",
    "orderType": "1"
  }

  res = requests.get(
    'http://front-gateway.mtime.com/library/movie/comment.api',
    params=params,
    headers=headers)

  comment_list = res.json()['data']['list']
  for i in comment_list:
    print("用户：", i['nickname'])
    print("评论：", i['content'])
  
  # 暂停一下，防止爬取太快被封
  time.sleep(1)

用户： 二哈排骨精
评论： 10年磨一剑，干翻迪斯尼
用户： JackieChen
评论： 一般般的动画电影，水军吹捧过头就不好了。真当有些人看了那么多年美漫和日漫 白看的？真当我们没见识？
用户： 麻匪118
评论： 中国式家庭典型代表，全村人的希望敖丙和我儿子再淘气也小可爱哪吒，史上最人性李靖，最可怜龙王，最接地气太乙金仙，最像男孩子的哪吒，哈哈哈，完美，必须二刷！
用户： 龚大叔
评论： 那些觉得不好看的，我只有用四川话回你：你晓得个锤子
用户： 天狼星上的眼镜猴
评论： 好好教育孩子，否则天雷来劈渣儿的时候都没人帮忙
用户： 银瞳
评论： 厅不错 音效画面都好 也没有小孩吵 电话响 问剧情 在国内看动画很久没有这么好的观影体验了 武戏分镜动作设计可圈可点 特别是江山图里四个人冻成冰球的段落很惊喜 可惜文戏让人如坐针毡 开场不到10分钟段
用户： jass
评论： 中国动画史上最悲怆的一幕：哪吒愤而举剑，割喉自杀。 今天的哪吒喊着 我命由我不由天，有点为赋新词强说愁的意思。艺术作品脱离了大时代背景，永远不可能成为真正意义上的经典。
用户： 心亦恋
评论： 休说苍天不由人，我命由我不由天。 视觉盛宴催旧泪，国漫希望谱新篇。
用户： 路见很平
评论： 都在说好，看了不过如此，风格不统一，过于糅杂，台词也尬得要命，从头到尾不舒服看不下去，编剧想让NPC怎样就怎样，完全不管人的行事逻辑，就最后的特效还行。总体不如同门的大圣归来，更不如彩条屋最出彩的大护
用户： 苗乐乐MiuMiu
评论： 每次大热电影出现都是言过其实了，跟风吹的人不必跟风黑的人少。故事上基本是胡编乱造毁名著，情感上可取的地方是没有过分煽情，但又有时幽默的不是地方，情感的拿捏比电影技术本身难多了。这个电影放到好莱坞水准衡
用户： 浪子回头金不换
评论： 前两天有位好友推荐看，加上这两天多方媒体好评报导，特别是时光网评分达到8.4，好奇释然，去了影院一观。看到了国产动画的进步，但是3D效果没做好，达不到真正的3D效果。剧情普通，没有惊喜感，从头看到尾令
用户： M_1504121653263072768
评论： 没想到一年多之后重返电影院看的第一部电影是国产动画片，还是一部这么好看的片子，好看到什么程度呢，我刚走出影厅，听到隔壁影厅哪吒的声音，我居然想溜进去再看一遍！整个故事情节编的好“圆”，没有觉得一丝

上面的代码获取了前 5 页的评论数据，如果我们需要，只要控制好爬取速度防止被封，完全可以将所有的短影评全部爬取下来。

除了评论之外，我们还能爬取某部电影的演员信息、评分等，只要掌握规律，想要的数据都可以得到。

如果你对电影的数据不感兴趣，你可以通过所学知识爬取你所感兴趣的网站。比如爬取租房网站，看看你所在地的房租是什么样的水平，哪里的房租最划算。爬取微博明星的评论，分析出哪些是水军等等。

虽然各个网站千差万别，但爬虫的基本思路与方法都是一样的。至此，动态网页的爬虫你已经掌握，大部分网站已经拦不住你，你可以爬取你想要的任何数据了。

除了更改 API 链接的 pageIndex 参数来翻页，我们还可以更改 pageSize 参数来增加每页数据，加快爬取速度。

比如将 pageNo 设为 2，pageSize 设为30，意思就每 30 条评论为一页，展现第 2 页。那我们就可以一次性获取第 31 到 60 条评论。

但是， pageSize 不能太大，否则可能会出错：可能不会返回内容，也可能只返回预设的上限数量，也可能会变成默认的 pageSize，每页数据反而更少。

接下来，请你根据讲解所学知识自己爬一遍评论，同时修改 pageSize 的值为 50，再获取前 3 页的用户昵称和评论试试。